# TPC4 - Analisador Léxico

Construir um analisador léxico para uma liguagem de query com a qual se podem escrever frases do
género:

 DBPedia: obras de Chuck Berry 
 

```sparql

select ?nome ?desc where { 
    ?s a dbo:MusicalArtist. 
    ?s foaf:name "Chuck Berry"@en . 
    ?w dbo:artist ?s. 
    ?w foaf:name ?nome. 
    ?w dbo:abstract ?desc 
} LIMIT 1000


In [26]:
tokens=[
    ('NEWLINE',r'\n'),
    ('SKIP',r'[ \t]+'),
    ('PREFIX',r'PREFIX\b'),
    ('SELECT',r'SELECT\b'),
    ('WHERE',r'WHERE\b'),
    ('OPTIONAL',r'OPTIONAL\b'),
    ('FILTER',r'FILTER\b'),
    ('VAR',r'\?[a-zA-Z_][\w]*'),
    ('URI',r'<[^>]*>'),
    ('IDENT',r':[a-zA-Z_][\w]*'),
    ('INT',r'\d+'),
    ('STRING',r'"[^\"]*"'),
    ('OP',r'[=!<>]+'),
    ('PUNCT',r'[{}.;,]'),
    ('ERROR',r'.')
]


In [27]:
import re

def tokenize(input):

    reconhecidos=[]
    linha=1 
    tokenreg = '|'.join(f'(?P<{name}>{pattern})' for name, pattern in tokens)
    val = re.finditer(tokenreg,input)
    for v in val:
        dic = v.groupdict()
        tipo = None
        valor = v.group()
        for key in dic:
            if dic[key]:
                tipo = key
                break
        if tipo == 'NEWLINE':
            linha += 1
            reconhecidos.append((tipo, valor, linha, v.span()))
        elif tipo != 'SKIP':
            reconhecidos.append((tipo, valor, linha, v.span()))
            
    return reconhecidos
    

input = """SELECT ?nome ?Tomas ?tudo  WHERE 
{
  ?Boa noite a :Pessoa ;
     :temIdade ?26 ;
     :eIrmaoDe ?Tomas .
}
"""
tokenize(input)

[('SELECT', 'SELECT', 1, (0, 6)),
 ('VAR', '?nome', 1, (7, 12)),
 ('VAR', '?Tomas', 1, (13, 19)),
 ('VAR', '?tudo', 1, (20, 25)),
 ('WHERE', 'WHERE', 1, (27, 32)),
 ('NEWLINE', '\n', 2, (33, 34)),
 ('PUNCT', '{', 2, (34, 35)),
 ('NEWLINE', '\n', 3, (35, 36)),
 ('VAR', '?Boa', 3, (38, 42)),
 ('ERROR', 'n', 3, (43, 44)),
 ('ERROR', 'o', 3, (44, 45)),
 ('ERROR', 'i', 3, (45, 46)),
 ('ERROR', 't', 3, (46, 47)),
 ('ERROR', 'e', 3, (47, 48)),
 ('ERROR', 'a', 3, (49, 50)),
 ('IDENT', ':Pessoa', 3, (51, 58)),
 ('PUNCT', ';', 3, (59, 60)),
 ('NEWLINE', '\n', 4, (60, 61)),
 ('IDENT', ':temIdade', 4, (66, 75)),
 ('ERROR', '?', 4, (76, 77)),
 ('INT', '26', 4, (77, 79)),
 ('PUNCT', ';', 4, (80, 81)),
 ('NEWLINE', '\n', 5, (81, 82)),
 ('IDENT', ':eIrmaoDe', 5, (87, 96)),
 ('VAR', '?Tomas', 5, (97, 103)),
 ('PUNCT', '.', 5, (104, 105)),
 ('NEWLINE', '\n', 6, (105, 106)),
 ('PUNCT', '}', 6, (106, 107)),
 ('NEWLINE', '\n', 7, (107, 108))]